In [9]:
import sys

sys.path.append('modules/')
from modules import goodreads, notion

# Provide the Goodreads URL to capture the metadata for
URL = "https://www.goodreads.com/book/show/55275019-machine-learning-design-patterns?ref=nav_sb_ss_1_111"

book = await goodreads.async_get_book_metadata(URL, verbose=True)

Name: Machine Learning Design Patterns: Solutions to Common Challenges in Data Preparation, Model Building, and MLOps
Image: https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1599583594i/55275019.jpg
Format: Paperback
Number of pages: 400
Language: English
ISBN: 9781098115784
Rating: 4.2
Number of ratings: 147
Number of reviews: 21
Authors: ['Valliappa Lakshmanan', 'Sara Robinson', 'Michael Munn']


In [12]:
notion.post_book_from_goodreads(book)

<Response [400]>